In [1]:
import cv2
import numpy as np
import winsound

line_start = (50, 250)
line_end = (400, 250)

center_x=0
center_y=0

video = cv2.VideoCapture("test_video2.mp4")
ret, frame = video.read()

cv2.line(frame, line_start, line_end, (0, 0, 255), 2)

# Select the region of interest (ROI) for tracking
# roi = cv2.selectROI(frame)
# print(roi)

roi=(185, 506, 40, 73)

# Create a mask for the ROI
mask1 = np.zeros_like(frame)
mask1[roi[1]:roi[1]+roi[3], roi[0]:roi[0]+roi[2]] = 255
#roi_mask=mask[roi[1]:roi[1]+roi[3], roi[0]:roi[0]+roi[2]]


# Set up the CAMShift tracker
tracker = cv2.TrackerCSRT_create()
tracker.init(frame, roi)   #initializing the tracker with the bounding box coordinates



while True:
   
    ret, frame = video.read()

    if not ret:
        break
        
        
   
    blur = cv2.GaussianBlur(frame, (5, 5), 0)
    #rec,thresh1=cv2.threshold(frame,127,255,cv2.THRESH_BINARY_INV)
    imgHSV=cv2.cvtColor(blur,cv2.COLOR_BGR2HSV)
    lower=np.array([4,52,16])
    upper=np.array([100,249,255])
    mask=cv2.inRange(imgHSV,lower,upper)
    
    carHSV=cv2.cvtColor(blur,cv2.COLOR_BGR2HSV)
    car_lower=np.array([0,55,153])
    car_upper=np.array([24,255,255])
    car_mask=cv2.inRange(carHSV,car_lower,car_upper)
    
    
    gray_mask1=cv2.cvtColor(mask1,cv2.COLOR_BGR2GRAY)
    
    # Update the tracker and get the new ROI
    success, roi = tracker.update(frame)

    # If the tracking is successful, draw the ROI on the frame
    if success:
        cv2.imshow("mask",mask)
        result_img=cv2.bitwise_and(frame,frame,mask=mask)
        car_img=cv2.bitwise_and(frame,frame,mask=car_mask)
        cv2.imshow("car Mask",car_mask)
        cv2.imshow("result_img",result_img)
        cv2.imshow("roi mask",mask1)
        
        # Unpack the ROI tuple and draw a rectangle
        x, y, w, h = [int(i) for i in roi]
        
        center_x = x + w // 2
        center_y = y + h // 2
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
   
        intersection = cv2.bitwise_and(mask,car_mask)
        
        num_white_pixels = cv2.countNonZero(intersection)
        
        if num_white_pixels>600:
            print(num_white_pixels)
            winsound.Beep(700,400)
        
    if line_start[0] <= center_x <= line_end[0] and line_start[1] <= center_y <= line_end[1]:
        crossing=True
        winsound.Beep(700,400)
        print(crossing)
         # Object has crossed the line, draw a circle around it   
        cv2.circle(frame, (center_x, center_y), 50, (0, 255, 0), -1)
    # Show the frame
    cv2.imshow("Tracking", frame)
    
    # Check if the user pressed 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
video.release()
cv2.destroyAllWindows()


607
608
True
True
